In [1]:
import os
os.environ['TFHUB_CACHE_DIR'] = '/home/djjindal/bert/script-learning'
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="2"

import tensorflow as tf

import read
import prepare_data
import input_builder

import importlib
import numpy as np
import model
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import *
from IPython.display import display
from IPython.html import widgets

BATCH_SIZE = 1
LEARNING_RATE = 2e-5
MAX_SEQ_LENGTH = 128

importlib.reload(input_builder)

model_dir = 'output_sentence'


W0511 19:18:21.466810 140637708261120 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0511 19:18:24.344674 140637708261120 tf_logging.py:115] Saver not created because there are no variables in the graph to restore
/home/djjindal/venvs/TF-1.3/lib/python3.6/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


# DATA

In [2]:
dataset = 'dataset/gw_extractions_no_rep_no_fin.pickle'
train_dataset = read.read_data_iterator(dataset)
features = list(prepare_data.tokenize_if_small_enough(train_dataset,sentences=True,no_context=False))
train_set = features[:int(0.8 * len(features))]
val_set = features[int(0.8 * len(features)):int(0.9*len(features))]
print(train_set[0])
len(train_set)

4810

In [8]:
from read import read_data_iterator as rdi
dat = rdi('dataset/gw_extractions_no_rep_no_fin.pickle')
da = list(dat)
print(da[0])

print("*****************************************")
dat2 = rdi('dataset/gw_extractions_with_rep.pickle')
da2 = list(dat2)
print(da2[0])

/home/djjindal/venvs/TF-1.3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: generator 'read_data_iterator' raised StopIteration
  This is separate from the ipykernel package so we can avoid doing imports until


{'entity': 'My father, a former movie exhibitor who taught me to respect\na theater like a cathedral', 'correct': 3, 'sentences': ['My father, a former movie exhibitor who taught me to respect\na theater like a cathedral, ran into that situation when\nhe saw Walk the Line in its opening week at\n', "He reported it to\na manager, who didn't know what to do except apologize\nand hand out passes for a return trip.\n\n\n", 'I got an e-mail from a 35-year veteran projectionist (he\nasked to remain anonymous) who generalized them as "teenage boys\npaid minimum wage, Burger King rejects, versus unionized, career professionals\n(who are) paid quite well, able to solve every technical\nand mechanical problem in a complex booth without having to\ncall in a service technician.'], 'triples': [(None, 'taught', 'me'), (None, 'reported', 'it'), (None, 'asked', 'to remain')], 'candidates': [(None, 'believe', 'the increasing demands'), (None, "'ve gotten", 'Part'), ('film', 'certainly is', None), (None

/home/djjindal/venvs/TF-1.3/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: generator 'read_data_iterator' raised StopIteration
  


# Model Objects

In [ ]:
run_config = tf.estimator.RunConfig(
    model_dir=model_dir,
    save_summary_steps=0,
    save_checkpoints_steps=0,
    log_step_count_steps=100)

model_fn = model.model_fn_builder(
  num_labels=5,
  learning_rate=LEARNING_RATE,
  num_train_steps=1,
  num_warmup_steps=1)

train_test_input_fn = input_builder.input_fn_builder(
    features=train_set,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False,
    candidates=5)

val_test_input_fn = input_builder.input_fn_builder(
    features=val_set,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False,
    candidates=5)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})

# PREDICT FUNCTION

In [ ]:
def predict(sentenecs, triples, candidates, entity):
    e_dict = dict()
    check_dataset = []
    e_dict['sentences'] = sentenecs
    e_dict['triples'] = triples
    e_dict['candidates'] = candidates
    e_dict['correct'] = 4
    e_dict['entity'] = entity
    check_dataset = [e_dict]
    predict_set = list(prepare_data.tokenize_if_small_enough(check_dataset, sentences=True,no_context=False))

    predict_input_fn = input_builder.input_fn_builder(
        features=predict_set,
        seq_length=MAX_SEQ_LENGTH,
        is_training=False,
        drop_remainder=False,
        candidates=5)
    predictions = estimator.predict(input_fn=predict_input_fn,yield_single_examples=False)

    for (i, prediction) in enumerate(predictions):
        arr = prediction['probabilities'][0]
        prob = [np.exp(arr[0]),np.exp(arr[1]),np.exp(arr[2]),np.exp(arr[3]),np.exp(arr[4])]
        total = np.sum(prob)
        max= np.max(prob)
        ec_dict = check_dataset[i]
        print("SENTENCE EVENT CHAIN", ec_dict['sentences'],"\n")
        print("TRIPLE EVENT CHAIN", ec_dict['triples'],"\n")
        print("CANDIDATES", ec_dict['candidates'],"\n")
        print("ENTITY", ec_dict['entity'],"\n")
        print(prob)
        print("PREDICTION", prediction['labels'], "with Probability", np.max(prob),"\n","\n")
        return prediction['labels']

def on_button_clicked(b):
    print(text1.value, text2.value, text3.value)
    print(predict(text1.value.split('), ('),[], text2.value.split('), ('), text3.value))    

# Some Examples

In [ ]:
dataset = list(read.read_data_iterator('dataset/gw_extractions_no_rep_no_fin.pickle'))
train_data = dataset[:int(0.8 * len(features))]
val_data = dataset[int(0.8 * len(features)):]

check_dataset = []

for i, ec_dict in zip(range(1000), val_data):
    check_dataset.append(ec_dict)
    
predict_set = list(prepare_data.tokenize_if_small_enough(check_dataset,sentences=True,no_context=False))
predict_input_fn = input_builder.input_fn_builder(
    features=predict_set,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False,
    candidates=5)

predictions = estimator.predict(input_fn=predict_input_fn,yield_single_examples=False)
predictions_list = list(predictions)

/home/djjindal/venvs/TF-1.3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: generator 'read_data_iterator' raised StopIteration
  """Entry point for launching an IPython kernel.


In [17]:
bad = []
for i, c in enumerate(check_dataset):
    r = list(prepare_data.tokenize_if_small_enough([c],sentences=True,no_context=False))
    if not r:
#         print(i)
        bad.append(i)
good_gt = [c for i, c in enumerate(check_dataset) if i not in bad]
pred_df = pd.DataFrame.from_dict({'predictions':predictions_list, 'dataset':good_gt})
print(len(good_gt))
print(len(predictions_list))

976
976


In [19]:
pred_df['pred_label'] = pred_df.predictions.apply(lambda x: x['labels'] - 1)
pred_df['gt_label'] = pred_df.dataset.apply(lambda x: x['correct'])
pred_df['correct_pred'] = pred_df.apply(lambda s: s.pred_label == s.gt_label, axis=1)
print(pred_df.correct_pred.sum()/len(pred_df.index))

0.011270491803278689


In [14]:
pred_df.to_pickle("./sentence1000_data.pkl")

In [21]:
pred_df

,predictions,dataset,pred_label,gt_label,correct_pred
0,"{'probabilities': [[-1.1421872, -1.7697272, -1...","{'entity': 'Bush', 'correct': 0, 'sentences': ...",-1,0,False
1,"{'probabilities': [[-1.080264, -1.5540415, -1....","{'entity': 'whose father', 'correct': 1, 'sent...",-1,1,False
2,"{'probabilities': [[-1.2451636, -1.5577687, -1...","{'entity': 'the authors', 'correct': 3, 'sente...",-1,3,False
3,"{'probabilities': [[-1.0362713, -1.5312059, -1...",{'entity': 'the Defense Threat Reduction Agenc...,-1,1,False
4,"{'probabilities': [[-1.1381506, -1.5359459, -1...","{'entity': 'Darlene', 'correct': 4, 'sentences...",-1,4,False
5,"{'probabilities': [[-1.033785, -1.4632381, -1....","{'entity': 'Ettinger', 'correct': 3, 'sentence...",-1,3,False
6,"{'probabilities': [[-1.1077057, -1.4326366, -1...","{'entity': 'Gov. George Pataki', 'correct': 3,...",-1,3,False
7,"{'probabilities': [[-1.1090736, -1.4665954, -1...","{'entity': 'Serge Savard', 'correct': 4, 'sent...",-1,4,False
8,"{'probabilities': [[-1.1038258, -1.5546149, -1...","{'entity': 'AOL', 'correct': 0, 'sentences': [...",-1,0,False
9,"{'probabilities': [[-1.1023157, -1.5003335, -1...","{'entity': 'Malick', 'correct': 3, 'sentences'...",-1,3,False


# DEMO

In [ ]:

# text1 = widgets.Text(description="Event Chain", width=200)
# text2 = widgets.Text(description="Candidates", width=200)
# text3 = widgets.Text(description="Entity", width=200)
# button = widgets.Button(description="Predict")
# button.on_click(on_button_clicked)
# display(text1)
# display(text2)
# display(text3)
# display(button)

# # ['We felt like fools, until our guide assured us that more intrepid tourists had met ghastly fates.', 'We heaved sighs of relief and reassured ourselves that no other activity on our trip would prove so nerve-wracking.', 'We had yet to reach Lake Atitlan, where a volcano beckoned us toward its cloud-shrouded apex.']
# # [('it', 'took', None), ('Guatemala City', 'broke', None), (None, 'had', 'Rickety'), ('September', 'averaging', None), (None, 'clog', 'the\npages')] 
# # us

# # [('john','ordered',None),('john','ate','food')]
# # [('john','left',None),('john','stays',None)]

# # [('john','ordered',None),('john','paid',None),('john','ate','food')]
# # [('john','left',None),('john','stays',None)]

# # [(None, 'took', 'phone call'), (None, 'had', 'something'), (None, 'began', 'to squint')] 
# # [(None, 'broadcast', '(the two previous record-holders'), ('the pennant', 'slammed', None), (None, "n't imagine", 'Maris'), (None, 'expressed', 'interest'), (None, 'turned', '68 _')] 

Text(value='', description='Event Chain')

Text(value='', description='Candidates')

Text(value='', description='Entity')

Button(description='Predict', style=ButtonStyle())